# Forced alignments data processing

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os
import subprocess
import shutil
from Bio import SeqIO
from Bio.Blast import NCBIXML
from collections import defaultdict

# Define functions


# Load data from JSON files into DataFrames
def load_json_to_df(filename):
    """
    Load data from a JSON file and return as a DataFrame.

    Parameters:
    filename (str): Path to the JSON file.

    Returns:
    pd.DataFrame: DataFrame containing data from the JSON file.
    """
    with open(filename, "r") as file:
        data = json.load(file)
    return pd.DataFrame(data)


def load_all_json_files(directory):
    """
    Loads all JSON files in a specified directory into pandas DataFrames, storing each
    DataFrame in a dictionary with keys derived from the file names.

    Parameters:
    directory (str): The directory from which to load JSON files.

    Returns:
    dict: A dictionary containing DataFrames. Keys are the file names without the '.json' extension.
    """
    data_frames = {}
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            file_path = os.path.join(directory, filename)
            data = load_json_to_df(file_path)
            # Use the file name without '.json' as the key
            key_name = filename[
                :-5
            ]  # Removes the last 5 characters, which should be ".json"
            data_frames[key_name] = pd.DataFrame(data)
    return data_frames


def parse_blast_xml_to_df(xml_file):
    """
    Parse the BLAST XML output file and extract alignment information.

    Parameters:
    xml_file (str): Path to the BLAST XML output file.

    Returns:
    pd.DataFrame: DataFrame containing parsed alignment information.
    """
    with open(xml_file) as result_handle:
        blast_records = NCBIXML.parse(result_handle)

        data = []
        for blast_record in blast_records:
            query_def = blast_record.query  # Accessing query definition
            for alignment in blast_record.alignments:
                hit_def = alignment.hit_def  # Accessing hit definition
                for hsp in alignment.hsps:
                    number_of_matches = hsp.identities
                    align_len = hsp.align_length
                    num_sims_per_base = (
                        number_of_matches / align_len
                    )  # Calculate the number of similarities per base

                    # Calculate the end coordinate
                    align_end = hsp.sbjct_start + align_len - 1

                    data.append(
                        {
                            "query_def": query_def,
                            "hit_def": hit_def,
                            "number_of_matches": number_of_matches,
                            "num_sims_per_base": num_sims_per_base,
                            "subject_alignment_seq": hsp.sbjct,
                            "query_alignment_seq": hsp.query,
                            "align_len": align_len,
                            "align_start": hsp.sbjct_start,
                            "align_end": align_end,  # Adding the end coordinate
                            "align_gap": hsp.gaps,  # Adding the gaps information
                        }
                    )
        return pd.DataFrame(data)


def calculate_mean_coverage_and_region(
    coverage, align_start, align_len, align_gap=0, use_align_gap=False
):
    """
    Calculate the mean coverage for a given region and return the coverage region.

    Parameters:
    coverage (list): List of coverage values.
    align_start (int): Start position of the alignment (one-based index).
    align_len (int): Length of the alignment.
    align_gap (int): Number of gaps in the alignment.
    use_align_gap (bool): Whether to consider alignment gaps in the calculation.

    Returns:
    tuple: Mean coverage of the specified region and the coverage region.
    """
    # Convert one-based align_start to zero-based start_index
    start_index = align_start - 1

    # Compute zero-based end_index, considering gaps if specified
    end_index = start_index + align_len - 1
    if use_align_gap:
        end_index -= align_gap

    # Extract the coverage region
    region_coverage = coverage[start_index : end_index + 1]  # end_index is inclusive
    mean_coverage = np.mean(region_coverage)
    return mean_coverage, region_coverage

## Load data:

In [3]:
T5_data_table = load_json_to_df(
    "../my_matlab_scripts_and_exports/summary_tables_json_files/T5_data_table.json"
)
T5samplesbamids = load_json_to_df(
    "../my_matlab_scripts_and_exports/summary_tables_json_files/T5samplesbamids.json"
)

## Merging `T5_data_table` and `T5samplesbamids`to have all the T5 data in one place in long format

In [4]:
# Convert T5samplesbamids to long format
id_vars = [
    "sample_name",
    "sample_id",
    "patient_id",
    "extraction",
    "virus",
    "viral_load",
    "concentration",
]
value_vars = [
    "rep_1_precapture",
    "rep_2_precapture",
    "rep_1_G14_capture",
    "rep_2_G14_capture",
    "rep_1_G1a_capture",
    "rep_2_G1a_capture",
    "T6_G16_capture",
]
T5samplesbamids_long = pd.melt(
    T5samplesbamids,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name="seq_method",
    value_name="run_id",
)

# Check for duplicates in T5_data_table to ensure that merging based on 'sample_id', 'seq_method', 'viral_load' is reasonable
duplicate_data_table = T5_data_table.duplicated(
    subset=["sample_id", "seq_method", "viral_load"], keep=False
)

# Check for duplicates in T5samplesbamids_long to ensure that merging based on 'sample_id', 'seq_method', 'viral_load' is reasonable
duplicate_samplesbamids = T5samplesbamids_long.duplicated(
    subset=["sample_id", "seq_method", "viral_load"], keep=False
)

# Check whether duplicates where found or not and go ahead with merging if there aren't any
if not duplicate_data_table.any() and not duplicate_samplesbamids.any():
    print(
        "No duplicates found, the chosen columns are appropriate for merging the dataframes."
    )

    # Merge the data tables on 'sample_id', 'seq_method', and 'viral_load'
    all_T5_data_long = pd.merge(
        T5_data_table,
        T5samplesbamids_long,
        on=["sample_id", "seq_method", "viral_load"],
        how="left",
    )  # include all rows from T5_data_table and match them with those from T5samplesbamids_long (I think this is the right option to choose but am slightly unsure)

    # Display the structure of the merged data
    print("\nHead of all_T5_data_long:")
    print(all_T5_data_long.head())
else:
    if duplicate_data_table.any():
        print(
            "Duplicates found in T5_data_table based on ['sample_id', 'seq_method', 'viral_load']:"
        )
        print(T5_data_table[duplicate_data_table])
    if duplicate_samplesbamids.any():
        print(
            "Duplicates found in T5samplesbamids_long based on ['sample_id', 'seq_method', 'viral_load']:"
        )
        print(T5samplesbamids_long[duplicate_samplesbamids])

No duplicates found, the chosen columns are appropriate for merging the dataframes.

Head of all_T5_data_long:
  sample_name_x         seq_method sample_id patient_id_x  viral_load  \
0       030-031   rep_1_precapture   030-031      030-031     19256.0   
1       030-031   rep_2_precapture   030-031      030-031     19256.0   
2       030-031  rep_1_G14_capture   030-031      030-031     19256.0   
3       030-031  rep_2_G14_capture   030-031      030-031     19256.0   
4       030-031  rep_1_G1a_capture   030-031      030-031     19256.0   

   num_total_reads  num_total_reads_cleaned  num_reads_mapped_to_human  \
0        7840314.0                7494972.0                  7265681.0   
1        8668044.0                8241300.0                  8006237.0   
2          78574.0                  77236.0                    64409.0   
3          83042.0                  81260.0                    69519.0   
4          19824.0                  19560.0                     4928.0   

   nu

## Filter high VL samples

The first thing happening in Azim's MATLAB code for plotting the capture efficiency figure is sorting `T5samplesbamids` by the highest viral loads and then taking just the first 15 rows (samples) with the highest viral loads (although one row is skipped). From what I understand from the comments in Azim's code, for these 15 samples, whole genome assemblies are available for both the direct MGS and capture with 1a probes approaches. Presumably, for the row that is skipped, no full assemblies were available (but I don't know). I replicate this by sorting and filtering `all_T5_data_long` accordingly.

In [5]:
# Sort the merged DataFrame by 'viral_load' in descending order
sorted_by_highest_viral_loads = all_T5_data_long.sort_values(
    by="viral_load", ascending=False
)

# Use this to select the top 15 high VL samples just like Azim did in his MATLAB code
# Select the top 15 unique sample_ids with the highest viral loads, skipping one row
top_15_sample_ids = pd.concat(
    [
        sorted_by_highest_viral_loads.drop_duplicates(subset="sample_id").head(14),
        sorted_by_highest_viral_loads.drop_duplicates(subset="sample_id").iloc[15:16],
    ]
)["sample_id"]

# Filter the combined data to include only the top 15 sample_ids
high_VL_samples = all_T5_data_long[
    all_T5_data_long["sample_id"].isin(top_15_sample_ids)
]

# Further filter to include only rows with 'seq_method' values 'rep_2_precapture' and 'rep_2_G1a_capture'.
high_VL_samples = high_VL_samples[
    high_VL_samples["seq_method"].isin(["rep_2_precapture", "rep_2_G1a_capture"])
]

# Keep only the columns that are important for the downstream analysis
high_VL_samples_filtered = high_VL_samples[
    [
        "sample_id",
        "seq_method",
        "run_id",
        "viral_load",
        "closest_ref",
        "coverage",
        "q50_major_type",
    ]
]

# Sort data frame and reset index values
high_VL_samples_filtered = high_VL_samples_filtered.sort_values(
    by=["viral_load", "sample_id", "seq_method"], ascending=False
).reset_index(drop=True)

# Display the structure of high_VL_samples_filtered
print("Head of high_VL_samples_filtered:")
print(high_VL_samples_filtered.head())

Head of high_VL_samples_filtered:
  sample_id         seq_method          run_id  viral_load  closest_ref  \
0   170-275   rep_2_precapture  wtchgD00001583   4854384.0    2b_D10988   
1   170-275  rep_2_G1a_capture  wtchgD00001748   4854384.0    2b_D10988   
2   161-074   rep_2_precapture  wtchgD00001571   4559808.0    2b_D10988   
3   161-074  rep_2_G1a_capture  wtchgD00001736   4559808.0    2b_D10988   
4   080-047   rep_2_precapture  wtchgD00001560   1795374.0  1b_EU781827   

                                            coverage q50_major_type  
0  [11, 11, 11, 11, 11, 12, 12, 12, 13, 13, 14, 4...             2b  
1  [42, 42, 42, 42, 52, 67, 84, 222, 367, 409, 52...             2b  
2  [21, 21, 21, 21, 23, 23, 23, 23, 23, 23, 24, 2...             2b  
3  [178, 192, 218, 232, 637, 855, 1177, 1960, 255...             2b  
4  [15, 15, 14, 13, 15, 13, 15, 56, 56, 69, 71, 7...             1b  


## Copy relevant reference and assembly sequences to a new folder to upload to BLAST

In [ ]:
# Function to copy the required assembly sequences to another directory
def copy_assembly_sequences(sample_row, output_dir):
    """
    Copy the direct MGS (HiSeq) de novo assembly genome sequence by looking up if seq_method is 'rep_2_precapture'.

    Parameters:
    sample_row (pd.Series): A row from the DataFrame containing sample data.
    output_dir (str): The directory where the sequences will be copied.
    """
    sample_id = sample_row["sample_id"]
    run_id = sample_row["run_id"]
    seq_method = sample_row["seq_method"]

    # Only process rows where seq_method is 'rep_2_precapture'
    if seq_method == "rep_2_precapture":
        # Copy the assembly sequence
        if run_id:
            hiseq_assembly_filename = f"../azim_data/Summaries/T5/{run_id}_closestdedup_vfat_fixed_assembly.fa"
            if os.path.exists(hiseq_assembly_filename):
                shutil.copy(
                    hiseq_assembly_filename,
                    os.path.join(output_dir, f"{run_id}_assembly.fa"),
                )
                print(f"Sample {sample_id}:")
                print(f"  HiSeq assembly: {run_id}")
                print()


output_dir = "../msa/input_to_msa"

# Iterate over all samples in high_VL_samples_filtered and copy the sequences
for _, row in high_VL_samples_filtered.iterrows():
    copy_assembly_sequences(row, output_dir)

print("Assembly sequences copied successfully to", output_dir)

Sample 170-275:
  HiSeq assembly: wtchgD00001583

Sample 161-074:
  HiSeq assembly: wtchgD00001571

Sample 080-047:
  HiSeq assembly: wtchgD00001560

Sample 080-016:
  HiSeq assembly: wtchgD00001558

Sample 070-290:
  HiSeq assembly: wtchgD00001555

Sample 070-425:
  HiSeq assembly: wtchgD00001557

Sample 170-270:
  HiSeq assembly: wtchgD00001582

Sample 170-238:
  HiSeq assembly: wtchgD00001575

Sample 080-054:
  HiSeq assembly: wtchgD00001562

Sample 170-243:
  HiSeq assembly: wtchgD00001576

Sample 170-254:
  HiSeq assembly: wtchgD00001577

Sample 080-051:
  HiSeq assembly: wtchgD00001561

Sample 160-223:
  HiSeq assembly: wtchgD00001567

Sample 080-034:
  HiSeq assembly: wtchgD00001559

Sample 070-337:
  HiSeq assembly: wtchgD00001556

Assembly sequences copied successfully to ../msa/input_to_msa


In [ ]:
# Function to copy the assembly sequence of the control samples (which the Geno1a probes were designed based upon) to another directory
def copy_assembly_sequences_of_control(sample_row, output_dir):
    """
    Copy the direct MGS (HiSeq) de novo assembly genome sequence of the control sample by looking up if seq_method is 'rep_2_precapture'.

    Parameters:
    sample_row (pd.Series): A row from the DataFrame containing sample data.
    output_dir (str): The directory where the sequences will be copied.
    """
    sample_id = sample_row["sample_name_x"]
    run_id = sample_row["run_id"]
    seq_method = sample_row["seq_method"]

    # Only process rows where seq_method is 'rep_2_precapture'
    if seq_method == "rep_2_precapture":
        if sample_id == "403-geno-1a-control":
            # Copy the assembly sequence of the control samples
            if run_id:
                hiseq_assembly_filename = f"../azim_data/Summaries/T5/{run_id}_closestdedup_vfat_fixed_assembly.fa"
                if os.path.exists(hiseq_assembly_filename):
                    shutil.copy(
                        hiseq_assembly_filename,
                        os.path.join(
                            output_dir, f"{run_id}_assembly_of_geno_1a_control.fa"
                        ),
                    )
                    print(f"Sample {sample_id}:")
                    print(f"  HiSeq assembly: {run_id}")
                    print()


output_dir = "../msa/input_to_msa"

# Iterate over all samples in high_VL_samples_filtered and copy the sequences
for _, row in all_T5_data_long.iterrows():
    copy_assembly_sequences_of_control(row, output_dir)

print("Assembly sequences of control sample copied successfully to", output_dir)

Sample 403-geno-1a-control:
  HiSeq assembly: wtchgD00001552

Assembly sequences of control sample copied successfully to ../msa/input_to_msa


## Combining sequences into a single FASTA file that I can easily upload to MAFFT

In [ ]:
def combine_fasta_files(input_dir, output_file):
    # List all fasta files in the input directory
    fasta_files = [
        f for f in os.listdir(input_dir) if f.endswith(".fa") or f.endswith(".fasta")
    ]

    # Open the output file in write mode
    with open(output_file, "w") as outfile:
        for fasta_file in fasta_files:
            fasta_path = os.path.join(input_dir, fasta_file)
            # Read each fasta file and write its content to the output file
            for record in SeqIO.parse(fasta_path, "fasta"):
                SeqIO.write(record, outfile, "fasta")


# Define the input directory and output file path
input_dir = "../msa/input_to_msa"
output_file = "../msa/input_to_msa/hiseq_assemblies_of_high_vl_samples_and_control.fa"

# Combine all fasta files into a single file
combine_fasta_files(input_dir, output_file)

print(f"All FASTA files from {input_dir} have been combined into {output_file}")

All FASTA files from ../msa/input_to_msa have been combined into ../msa/input_to_msa/hiseq_assemblies_of_high_vl_samples_and_control.fa


## Length of whole genome sequences
### Before MSA

In [ ]:
def analyze_fasta_lengths(fasta_file):
    lengths = []

    # Read sequences from the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        lengths.append(len(record.seq))
        print(f"Sequence: {record.id}, Length: {len(record.seq)}")

    if lengths:
        shortest = min(lengths)
        longest = max(lengths)
        print(f"\nShortest genome length: {shortest}")
        print(f"Longest genome length: {longest}")
        print(
            f"\nThe lengths of the genomes range from {shortest} to {longest} base pairs."
        )
    else:
        print("No sequences found in the file.")


# Specify the path to your FASTA file
before_msa = "../msa/input_to_msa/hiseq_assemblies_of_high_vl_samples_and_control.fa"

# Run the analysis
analyze_fasta_lengths(before_msa)

Sequence: wtchgD00001556_closestdedup_vicunavfat, Length: 9378
Sequence: wtchgD00001556_closestdedup_vicunavfat, Length: 9378
Sequence: wtchgD00001571_closestdedup_vicunavfat, Length: 9485
Sequence: wtchgD00001561_closestdedup_vicunavfat, Length: 9410
Sequence: wtchgD00001583_closestdedup_vicunavfat, Length: 9478
Sequence: wtchgD00001562_closestdedup_vicunavfat, Length: 9403
Sequence: wtchgD00001577_closestdedup_vicunavfat, Length: 9410
Sequence: wtchgD00001559_closestdedup_vicunavfat, Length: 9386
Sequence: wtchgD00001555_closestdedup_vicunavfat, Length: 9408
Sequence: wtchgD00001552_closestdedup_vicunavfat, Length: 9423
Sequence: wtchgD00001567_closestdedup_vicunavfat, Length: 9418
Sequence: wtchgD00001560_closestdedup_vicunavfat, Length: 9411
Sequence: wtchgD00001575_closestdedup_vicunavfat, Length: 9411
Sequence: wtchgD00001557_closestdedup_vicunavfat, Length: 9382
Sequence: wtchgD00001576_closestdedup_vicunavfat, Length: 9411
Sequence: wtchgD00001558_closestdedup_vicunavfat, Lengt

### After MSA

In [ ]:
after_msa = "../msa/output_of_msa/mafft-I20240610-125150-0506-42658743-p1m.aln-fasta"

analyze_fasta_lengths(after_msa)

Sequence: wtchgD00001556_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001562_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001552_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001561_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001576_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001577_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001567_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001558_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001555_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001560_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001575_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001559_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001557_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001571_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001583_closestdedup_vicunavfat, Length: 9530
Sequence: wtchgD00001582_closestdedup_vicunavfat, Lengt

## Parsing megaBLAST output of querying Geno1a probes to control sample
Using megaBLAST to get the start and end coordinates of each Geno1a probe aligned to the HiSeq assembly of the sample the probes were designed based upon

In [ ]:
# Path to your BLAST XML output file
xml_file = "../msa/blast_to_control/megablast_geno_1a_probes_to_control_sample_6DUCZ3UB114-Alignment.xml"

# Parse the BLAST XML output
megablast_results_df = parse_blast_xml_to_df(xml_file)

# Display head of blast_results_df to understand the structure
print("Head of megablast_results_df:\n", megablast_results_df.head())

Head of megablast_results_df:
         query_def                                 hit_def  number_of_matches  \
0  Geno1a_probe_1  wtchgD00001552_closestdedup_vicunavfat                120   
1  Geno1a_probe_2  wtchgD00001552_closestdedup_vicunavfat                118   
2  Geno1a_probe_3  wtchgD00001552_closestdedup_vicunavfat                120   
3  Geno1a_probe_4  wtchgD00001552_closestdedup_vicunavfat                120   
4  Geno1a_probe_5  wtchgD00001552_closestdedup_vicunavfat                120   

   num_sims_per_base                              subject_alignment_seq  \
0           1.000000  CCAGCCCCCTGATGGGGGCGACACTCCACCATGAATCACTCCCCTG...   
1           0.983333  CTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTG...   
2           1.000000  CCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTA...   
3           1.000000  GGACGACCGGGTCCTTTCTTGGATCAACCCGCTCAATGCCTGGAGA...   
4           1.000000  CGCAAGACTGCTAGCCGAGTAGTGTTGGGTCGCGAAAGGCCTTGTG...   

                                 quer

## Trying BLASTn instead of megaBLAST

In [ ]:
# Path to your BLAST XML output file
xml_file = "../msa/blast_to_control/blastn_geno_1a_probes_to_control_sample_6DV9XG8D114-Alignment.xml"

# Parse the BLAST XML output
blast_results_df = parse_blast_xml_to_df(xml_file)

# Display head of blast_results_df to understand the structure
print("Head of blast_results_df:\n", blast_results_df.head())

Head of blast_results_df:
         query_def                                 hit_def  number_of_matches  \
0  Geno1a_probe_1  wtchgD00001552_closestdedup_vicunavfat                120   
1  Geno1a_probe_2  wtchgD00001552_closestdedup_vicunavfat                118   
2  Geno1a_probe_3  wtchgD00001552_closestdedup_vicunavfat                120   
3  Geno1a_probe_4  wtchgD00001552_closestdedup_vicunavfat                120   
4  Geno1a_probe_5  wtchgD00001552_closestdedup_vicunavfat                120   

   num_sims_per_base                              subject_alignment_seq  \
0           1.000000  CCAGCCCCCTGATGGGGGCGACACTCCACCATGAATCACTCCCCTG...   
1           0.983333  CTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTG...   
2           1.000000  CCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTA...   
3           1.000000  GGACGACCGGGTCCTTTCTTGGATCAACCCGCTCAATGCCTGGAGA...   
4           1.000000  CGCAAGACTGCTAGCCGAGTAGTGTTGGGTCGCGAAAGGCCTTGTG...   

                                 query_al

### Only extracting the top alignment hits from BLASTn
By using BLASTn, I got more than 157 alignment hits, so here I extract only the top (best) alignment hits for each probe

In [ ]:
def parse_top_alignment_hits_blast_xml_to_df(xml_file):
    """
    Parse the BLAST XML output file and extract the top alignment hit information for each query.

    Parameters:
    xml_file (str): Path to the BLAST XML output file.

    Returns:
    pd.DataFrame: DataFrame containing parsed alignment information for the top hit of each query.
    """
    with open(xml_file) as result_handle:
        blast_records = NCBIXML.parse(result_handle)

        data = []
        for blast_record in blast_records:
            query_def = blast_record.query  # Accessing query definition
            if blast_record.alignments:  # Check if there are any alignments
                # Process only the top alignment
                top_alignment = blast_record.alignments[0]
                hit_def = top_alignment.hit_def  # Accessing hit definition
                for hsp in top_alignment.hsps:
                    number_of_matches = hsp.identities
                    align_len = hsp.align_length
                    num_sims_per_base = (
                        number_of_matches / align_len
                    )  # Calculate the number of similarities per base

                    # Calculate the end coordinate
                    align_end = hsp.sbjct_start + align_len - 1

                    data.append(
                        {
                            "query_def": query_def,
                            "hit_def": hit_def,
                            "number_of_matches": number_of_matches,
                            "num_sims_per_base": num_sims_per_base,
                            "subject_sequence": hsp.sbjct,
                            "query_sequence": hsp.query,
                            "align_len": align_len,
                            "align_start": hsp.sbjct_start,
                            "align_end": align_end,  # Adding the end coordinate
                            "align_gap": hsp.gaps,  # Adding the gaps information
                        }
                    )
                    break  # Only process the first HSP for the top alignment
        return pd.DataFrame(data)


# Path to your BLAST XML output file
xml_file = "../msa/blast_to_control/blastn_geno_1a_probes_to_control_sample_6DV9XG8D114-Alignment.xml"

# Parse the BLAST XML output
blastn_top_results_df = parse_top_alignment_hits_blast_xml_to_df(xml_file)

# Display head of blast_results_df to understand the structure
print("Head of blastn_top_results_df:\n", blastn_top_results_df.head())

Head of blastn_top_results_df:
         query_def                                 hit_def  number_of_matches  \
0  Geno1a_probe_1  wtchgD00001552_closestdedup_vicunavfat                120   
1  Geno1a_probe_2  wtchgD00001552_closestdedup_vicunavfat                118   
2  Geno1a_probe_3  wtchgD00001552_closestdedup_vicunavfat                120   
3  Geno1a_probe_4  wtchgD00001552_closestdedup_vicunavfat                120   
4  Geno1a_probe_5  wtchgD00001552_closestdedup_vicunavfat                120   

   num_sims_per_base                                   subject_sequence  \
0           1.000000  CCAGCCCCCTGATGGGGGCGACACTCCACCATGAATCACTCCCCTG...   
1           0.983333  CTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTG...   
2           1.000000  CCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTA...   
3           1.000000  GGACGACCGGGTCCTTTCTTGGATCAACCCGCTCAATGCCTGGAGA...   
4           1.000000  CGCAAGACTGCTAGCCGAGTAGTGTTGGGTCGCGAAAGGCCTTGTG...   

                                    

## Use Nick's code to do genome coordinate conversion via MSA

### Define functions:
I let GPT-4o add some further comments to illustrate what the functions do. I am unsure how accurate these descriptions are.

In [ ]:
"""
Note that this uses 1-based numbering to match genbank files

Take a read sequence eg [1,2,3,4,5] and convert the numbers to be the positions of the matched reference eg [1,2,3,5,6] using 
the multiple sequence alignment (msa) with the matched reference

Then convert the matched reference positions to the reference msa positions

Then convert reference msa positions to the base reference

See the contrived example where the resulting alignment isn't optimal and information is lost in the gaps between the alignments
"""


class GappedSequenceObject(object):
    """
    A class to represent a gapped sequence and provide methods to convert between
    gapped and ungapped positions.

    Attributes:
    sequence (str): The gapped sequence.
    id (str): An optional name for the sequence.
    _gapped_to_ungapped_position_look_up_list (list): Cached list for gapped to ungapped position conversion.
    _ungapped_to_gapped_position_look_up_list (list): Cached list for ungapped to gapped position conversion.
    """

    def __init__(self, genome_string, name=None):
        self.sequence = genome_string
        self.id = name
        self._gapped_to_ungapped_position_look_up_list = None
        self._ungapped_to_gapped_position_look_up_list = None

    def gapped_to_ungapped_position_look_up_list_get(self):
        """
        Generate or return a cached list that converts gapped positions to ungapped positions.

        Returns:
        list: A list where each index represents a gapped position and the value at that index
        is the corresponding ungapped position or None if it's a gap.
        """
        if self._gapped_to_ungapped_position_look_up_list is None:
            ungapped_position = 1
            self._gapped_to_ungapped_position_look_up_list = [0]  # start from 1
            for nucleotide_object in self.sequence:
                if nucleotide_object == "-":
                    self._gapped_to_ungapped_position_look_up_list.append(None)
                else:
                    self._gapped_to_ungapped_position_look_up_list.append(
                        ungapped_position
                    )
                    ungapped_position += 1
        return self._gapped_to_ungapped_position_look_up_list

    gapped_to_ungapped_position_look_up_list = property(
        gapped_to_ungapped_position_look_up_list_get
    )

    def ungapped_to_gapped_position_look_up_list_get(self):
        """
        Generate or return a cached list that converts ungapped positions to gapped positions.

        Returns:
        list: A list where each index represents an ungapped position and the value at that index
        is the corresponding gapped position.
        """
        if self._ungapped_to_gapped_position_look_up_list is None:
            gapped_position = 1
            self._ungapped_to_gapped_position_look_up_list = [0]  # start from 1
            for nucleotide_object in self.sequence:
                if nucleotide_object == "-":
                    pass
                else:
                    self._ungapped_to_gapped_position_look_up_list.append(
                        gapped_position
                    )
                gapped_position += 1
        return self._ungapped_to_gapped_position_look_up_list

    ungapped_to_gapped_position_look_up_list = property(
        ungapped_to_gapped_position_look_up_list_get
    )

    # def return_first_valid_position_gapped_to_ungapped_3_prime_direction(self, gapped_position):
    #     reference_position = None
    #     while reference_position is None:
    #         if gapped_position == len(self.gapped_to_ungapped_position_look_up_list):
    #             return len(self.sequence.replace("-", ""))   # reached end
    #         reference_position = self.gapped_to_ungapped_position_look_up_list[gapped_position]
    #         gapped_position += 1
    #     return reference_position
    #
    # def return_first_valid_position_gapped_to_ungapped_5_prime_direction(self, gapped_position):
    #     reference_position = None
    #     while reference_position is None:
    #         if gapped_position == 1:  # reached start
    #             return 1
    #         reference_position = self.gapped_to_ungapped_position_look_up_list[gapped_position]
    #         # print("reference_position", reference_position)
    #         gapped_position -= 1
    #     return reference_position


def convert_read_position_list_to_matched_reference_position_list(
    matched_reference_msa_read_sequence,
    matched_reference_msa_reference_sequence,
    matched_reference_msa_alignment_start,
):
    """
    Convert read positions to matched reference positions using MSA.

    Parameters:
    matched_reference_msa_read_sequence (str): The read sequence aligned in MSA.
    matched_reference_msa_reference_sequence (str): The reference sequence aligned in MSA.
    matched_reference_msa_alignment_start (int): The starting position of alignment.

    Returns:
    list: A list of positions in the matched reference corresponding to each position in the read.
    """
    matched_reference_position_list = []
    matched_reference_sequence_offset = matched_reference_msa_alignment_start

    for read_nucleotide, reference_nucleotide in zip(
        matched_reference_msa_read_sequence, matched_reference_msa_reference_sequence
    ):
        if read_nucleotide == "-":
            matched_reference_sequence_offset += 1
        else:
            if reference_nucleotide == "-":
                matched_reference_position_list.append(
                    None
                )  # reference must have a nucleotide
            else:
                matched_reference_position_list.append(
                    matched_reference_sequence_offset
                )
                matched_reference_sequence_offset += 1
    return matched_reference_position_list


def convert_matched_reference_position_list_to_reference_msa_position_list(
    matched_reference_position_list, matched_reference_reference_msa_sequence
):
    """
    Convert matched reference positions to MSA positions.

    Parameters:
    matched_reference_position_list (list): List of positions in the matched reference.
    matched_reference_reference_msa_sequence (str): Reference sequence in the MSA.

    Returns:
    list: A list of positions in the MSA corresponding to each position in the matched reference.
    """
    matched_reference_reference_msa_sequence_object = GappedSequenceObject(
        matched_reference_reference_msa_sequence
    )

    reference_msa_position_list = []
    for position in matched_reference_position_list:
        if position is None:
            reference_msa_position_list.append(None)
        else:
            try:
                reference_msa_position_list.append(
                    matched_reference_reference_msa_sequence_object.ungapped_to_gapped_position_look_up_list[
                        position
                    ]
                )
            except IndexError:
                break  # reached end of read match sequence
    return reference_msa_position_list


def convert_reference_msa_position_list_to_base_reference_position_list(
    reference_msa_position_list, base_reference_reference_msa_sequence
):
    """
    Convert MSA positions to base reference positions.

    Parameters:
    reference_msa_position_list (list): List of positions in the reference MSA.
    base_reference_reference_msa_sequence (str): The base reference sequence in the MSA.

    Returns:
    list: A list of positions in the base reference corresponding to each position in the MSA.
    """
    base_reference_reference_msa_sequence_object = GappedSequenceObject(
        base_reference_reference_msa_sequence
    )

    base_reference_read_position_list = []
    for position in reference_msa_position_list:
        if position is None:
            base_reference_read_position_list.append(None)
        else:
            base_reference_read_position_list.append(
                base_reference_reference_msa_sequence_object.gapped_to_ungapped_position_look_up_list[
                    position
                ]
            )
    return base_reference_read_position_list


def return_base_reference_msa_sequences(
    read_base_reference_position_list, read_sequence, base_reference_sequence
):
    """
    Generate aligned sequences for read and base reference.

    Parameters:
    read_base_reference_position_list (list): Positions of the read in the base reference.
    read_sequence (str): The sequence of the read.
    base_reference_sequence (str): The sequence of the base reference.

    Returns:
    tuple: Aligned sequences of the read and the base reference.
    """
    read_nucleotide_list = []
    reference_nucleotide_list = []  # list(ref1[:read_reference_position_list[0]-1])
    previous_reference_position = 0
    i = 0
    while i < len(read_base_reference_position_list):
        if read_base_reference_position_list[i] is not None:
            while (
                read_base_reference_position_list[i] != previous_reference_position + 1
            ):
                read_nucleotide_list.append("-")
                reference_nucleotide_list.append(
                    base_reference_sequence[previous_reference_position]
                )
                previous_reference_position += 1
            if i == len(read_base_reference_position_list):
                break
            nucleotide = base_reference_sequence[
                read_base_reference_position_list[i] - 1
            ]
            read_nucleotide_list.append(read_sequence[i])
            reference_nucleotide_list.append(nucleotide)
            previous_reference_position += 1
        else:
            read_nucleotide_list.append(read_sequence[i])
            reference_nucleotide_list.append("-")
        i += 1
    base_reference_msa_read_sequence = "".join(str(x) for x in read_nucleotide_list)
    base_reference_msa_reference_sequence = "".join(
        str(x) for x in reference_nucleotide_list
    )
    return base_reference_msa_read_sequence, base_reference_msa_reference_sequence

### Extract control sample sequence from MAFFT MSA

In [ ]:
def print_all_sequence_ids_from_fasta(fasta_file):
    """
    Print the names of all sequence IDs in a FASTA file.

    Parameters:
    fasta_file (str): Path to the FASTA file containing multiple sequences.
    """
    for record in SeqIO.parse(fasta_file, "fasta"):
        print(record.id)


msa = "../msa/output_of_msa/mafft-I20240610-125150-0506-42658743-p1m.aln-fasta"

print_all_sequence_ids_from_fasta(msa)

wtchgD00001556_closestdedup_vicunavfat
wtchgD00001562_closestdedup_vicunavfat
wtchgD00001552_closestdedup_vicunavfat
wtchgD00001561_closestdedup_vicunavfat
wtchgD00001576_closestdedup_vicunavfat
wtchgD00001577_closestdedup_vicunavfat
wtchgD00001567_closestdedup_vicunavfat
wtchgD00001558_closestdedup_vicunavfat
wtchgD00001555_closestdedup_vicunavfat
wtchgD00001560_closestdedup_vicunavfat
wtchgD00001575_closestdedup_vicunavfat
wtchgD00001559_closestdedup_vicunavfat
wtchgD00001557_closestdedup_vicunavfat
wtchgD00001571_closestdedup_vicunavfat
wtchgD00001583_closestdedup_vicunavfat
wtchgD00001582_closestdedup_vicunavfat


In [ ]:
# the genome coordinate conversion functions work on text strings, so I first extract the raw sequence string from the fasta files
def extract_sequence_from_fasta_file_as_text(fasta_file, sequence_id):
    """
    Extract a specific sequence from a FASTA file as plain text.

    Parameters:
    fasta_file (str): Path to the FASTA file containing multiple sequences.
    sequence_id (str): The ID of the sequence to extract.

    Returns:
    str: The sequence as a plain text string, or None if not found.
    """
    for record in SeqIO.parse(fasta_file, "fasta"):
        if record.id == sequence_id:
            return str(record.seq)
    return None


# this is the names of the direct MGS sequencing run on the control sample that should contain the genome sequence which the Geno1a probes are designed based upon
control_sample_sequence_id = "wtchgD00001552_closestdedup_vicunavfat"

control_sequence_from_msa = extract_sequence_from_fasta_file_as_text(
    msa, control_sample_sequence_id
)

# print(
#     f"Sequence from MSA for {control_sample_sequence_id} (gaps expected):\n{control_sequence_from_msa}"
# )

control_sequence_from_hiseq_assembly_file_path = (
    "../msa/input_to_msa/wtchgD00001552_assembly_of_geno_1a_control.fa"
)

control_sequence_from_hiseq_assembly = extract_sequence_from_fasta_file_as_text(
    control_sequence_from_hiseq_assembly_file_path, control_sample_sequence_id
)

# print(
#     f"\nSequence from HiSeq assembly for {control_sample_sequence_id} (should be gapless):\n{control_sequence_from_hiseq_assembly}"
# )

### Convert coordinates between gapless control sequence and gapped control sequence from MSA
#### Working through example for one probe in one sample

In [ ]:
# Before applying the coordinate conversion code to all probe alignments in all high VL samples, first try it out with one probe alignment ("read") in one test sample
test_sample_sequence_id = "wtchgD00001556_closestdedup_vicunavfat"

test_sample_sequence_from_msa = extract_sequence_from_fasta_file_as_text(
    msa, test_sample_sequence_id
)

# print(
#     f"Sequence from MSA for {test_sample_sequence_id}:\n{test_sample_sequence_from_msa}"
# )

base_reference_reference_msa_sequence = test_sample_sequence_from_msa
matched_reference_reference_msa_sequence = control_sequence_from_msa

matched_reference_msa_reference_sequence = "CCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTACACCGGAATTGCCAGGACGACCGGGTCCTTTCTTGGATCAACCCGCTCAATGCCTGGAGATTTGGGCGTGCCCC"
matched_reference_msa_alignment_start = 120

matched_reference_msa_read_sequence = "CCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTACACCGGAATTGCCAGGACGACCGGGTCCTTTCTTGGATCAACCCGCTCAATGCCTGGAGATTTGGGCGTGCCCC"

read_sequence = matched_reference_msa_read_sequence.replace("-", "")
base_reference_sequence = base_reference_reference_msa_sequence.replace("-", "")

matched_reference_position_list = (
    convert_read_position_list_to_matched_reference_position_list(
        matched_reference_msa_read_sequence,
        matched_reference_msa_reference_sequence,
        matched_reference_msa_alignment_start,
    )
)
print("\nmatched_reference_position_list:", matched_reference_position_list)


reference_msa_position_list = (
    convert_matched_reference_position_list_to_reference_msa_position_list(
        matched_reference_position_list, matched_reference_reference_msa_sequence
    )
)
print("\nreference_msa_position_list:", reference_msa_position_list)

base_reference_read_position_list = (
    convert_reference_msa_position_list_to_base_reference_position_list(
        reference_msa_position_list, base_reference_reference_msa_sequence
    )
)
print("\nbase_reference_read_position_list:", base_reference_read_position_list)

base_reference_msa_read_sequence, base_reference_msa_reference_sequence = (
    return_base_reference_msa_sequences(
        base_reference_read_position_list, read_sequence, base_reference_sequence
    )
)
print("\nbase_reference_msa_read_sequence:     ", base_reference_msa_read_sequence)
print("base_reference_msa_reference_sequence:", base_reference_msa_reference_sequence)


matched_reference_position_list: [120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239]

reference_msa_position_list: [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,

#### For all samples

In [ ]:
# Function to extract all sequence IDs from an MSA file
def get_sequence_ids_from_msa(msa_file, exclude_id=None):
    """
    Get all sequence IDs from an MSA file, excluding a specific ID if provided.

    Parameters:
    msa_file (str): Path to the MSA file.
    exclude_id (str): Sequence ID to exclude from the list.

    Returns:
    list: List of sequence IDs.
    """
    sequence_ids = []
    for record in SeqIO.parse(msa_file, "fasta"):
        if record.id != exclude_id:
            sequence_ids.append(record.id)
    return sequence_ids


# Function to process all probes for all samples
def process_all_samples(
    probes_df, msa_file, hiseq_assembly_file, control_sample_sequence_id
):
    """
    Process all probes for all samples and store the results in a DataFrame.

    Parameters:
    probes_df (pd.DataFrame): DataFrame containing BLASTn results for the probes.
    msa_file (str): Path to the MSA file.
    hiseq_assembly_file (str): Path to the HiSeq assembly file.
    control_sample_sequence_id (str): The ID of the control sample sequence.

    Returns:
    pd.DataFrame: DataFrame containing the results for all probes in all samples.
    """
    # Extract all sequence IDs from the MSA file, excluding the control sequence ID
    sample_ids = get_sequence_ids_from_msa(
        msa_file, exclude_id=control_sample_sequence_id
    )

    # Extract the control sequences from the MSA and HiSeq assembly files
    control_sequence_from_msa = extract_sequence_from_fasta_file_as_text(
        msa_file, control_sample_sequence_id
    )
    control_sequence_from_hiseq_assembly = extract_sequence_from_fasta_file_as_text(
        hiseq_assembly_file, control_sample_sequence_id
    )

    all_results = []

    # Iterate through each sample
    for sample_sequence_id in sample_ids:
        print(f"Processing sample: {sample_sequence_id}")

        # Extract the sample sequence from the MSA file
        sample_sequence_from_msa = extract_sequence_from_fasta_file_as_text(
            msa_file, sample_sequence_id
        )

        # Prepare the base reference sequence
        base_reference_sequence = sample_sequence_from_msa.replace("-", "")

        # Process each probe for this sample
        for index, row in probes_df.iterrows():
            matched_reference_msa_read_sequence = row["query_sequence"]
            matched_reference_msa_reference_sequence = row["subject_sequence"]
            matched_reference_msa_alignment_start = row["align_start"]

            read_sequence = matched_reference_msa_read_sequence.replace("-", "")

            # Convert coordinates
            matched_reference_position_list = (
                convert_read_position_list_to_matched_reference_position_list(
                    matched_reference_msa_read_sequence,
                    matched_reference_msa_reference_sequence,
                    matched_reference_msa_alignment_start,
                )
            )

            reference_msa_position_list = (
                convert_matched_reference_position_list_to_reference_msa_position_list(
                    matched_reference_position_list, control_sequence_from_msa
                )
            )

            base_reference_read_position_list = (
                convert_reference_msa_position_list_to_base_reference_position_list(
                    reference_msa_position_list, sample_sequence_from_msa
                )
            )

            base_reference_msa_read_sequence, base_reference_msa_reference_sequence = (
                return_base_reference_msa_sequences(
                    base_reference_read_position_list,
                    read_sequence,
                    base_reference_sequence,
                )
            )

            # Store the results
            all_results.append(
                {
                    "sample_id": sample_sequence_id,
                    "probe_id": row["query_def"],
                    "probe_sequence": row["query_sequence"],
                    "matched_reference_position_list": matched_reference_position_list,
                    "reference_msa_position_list": reference_msa_position_list,
                    "base_reference_read_position_list": base_reference_read_position_list,
                    "base_reference_msa_read_sequence": base_reference_msa_read_sequence,
                    "base_reference_msa_reference_sequence": base_reference_msa_reference_sequence,
                }
            )

    return pd.DataFrame(all_results)


# Process all probes for all samples and store the results in a DataFrame
msa_file = "../msa/output_of_msa/mafft-I20240610-125150-0506-42658743-p1m.aln-fasta"
hiseq_assembly_file = (
    "../msa/input_to_msa/wtchgD00001552_assembly_of_geno_1a_control.fa"
)
control_sample_sequence_id = "wtchgD00001552_closestdedup_vicunavfat"

all_probes_results_df = process_all_samples(
    blastn_top_results_df, msa_file, hiseq_assembly_file, control_sample_sequence_id
)

# Display the results
print("\nHead of all_probes_results_df:", all_probes_results_df.head())

Processing sample: wtchgD00001556_closestdedup_vicunavfat
Processing sample: wtchgD00001562_closestdedup_vicunavfat
Processing sample: wtchgD00001561_closestdedup_vicunavfat
Processing sample: wtchgD00001576_closestdedup_vicunavfat
Processing sample: wtchgD00001577_closestdedup_vicunavfat
Processing sample: wtchgD00001567_closestdedup_vicunavfat
Processing sample: wtchgD00001558_closestdedup_vicunavfat
Processing sample: wtchgD00001555_closestdedup_vicunavfat
Processing sample: wtchgD00001560_closestdedup_vicunavfat
Processing sample: wtchgD00001575_closestdedup_vicunavfat
Processing sample: wtchgD00001559_closestdedup_vicunavfat
Processing sample: wtchgD00001557_closestdedup_vicunavfat
Processing sample: wtchgD00001571_closestdedup_vicunavfat
Processing sample: wtchgD00001583_closestdedup_vicunavfat
Processing sample: wtchgD00001582_closestdedup_vicunavfat

Head of all_probes_results_df:                                 sample_id        probe_id  \
0  wtchgD00001556_closestdedup_vicuna

### Extract regions of the genomes that the probes were force-aligned to

In [ ]:
# Method 1: Using a slighlty hacky approach of counting the number of hyphens at the beginning of base_reference_msa_read_sequence and removing that number from the beginning of base_reference_msa_reference_sequence
def extract_alignment_region_hyphen(row):
    # Count leading hyphens only
    hyphen_count = 0
    for char in row["base_reference_msa_read_sequence"]:
        if char == "-":
            hyphen_count += 1
        else:
            break

    start_index = hyphen_count
    end_index = start_index + 120
    sequence = row["base_reference_msa_reference_sequence"][start_index:end_index]

    return sequence


all_probes_results_df["base_reference_alignment_region_hyphen"] = (
    all_probes_results_df.apply(extract_alignment_region_hyphen, axis=1)
)


# Method 2: Taking the first value in base_reference_read_position_list -1 and removing that number of bases from the beginning of base_reference_msa_reference_sequence
def extract_alignment_region_start_index(row):
    base_reference_read_position_list = row["base_reference_read_position_list"]

    # Remove commas and ignore leading None values
    base_reference_read_position_list = [
        pos for pos in base_reference_read_position_list if pos != ","
    ]

    none_count = 0
    for pos in base_reference_read_position_list:
        if pos is None:
            none_count += 1
        else:
            break

    first_valid_position = base_reference_read_position_list[none_count]
    if first_valid_position is not None:
        start_index = first_valid_position - 1  # Convert to 0-based index
        end_index = start_index + 120

        # Ensure start_index is not negative
        start_index = max(0, start_index)

        # Ensure end_index does not exceed the sequence length
        end_index = min(len(row["base_reference_msa_reference_sequence"]), end_index)

        sequence = row["base_reference_msa_reference_sequence"][start_index:end_index]

        return sequence
    return None


all_probes_results_df["base_reference_alignment_region_start_index"] = (
    all_probes_results_df.apply(extract_alignment_region_start_index, axis=1)
)

# Sanity check: Verify if the sequences in both methods are identical
discrepancies = all_probes_results_df[
    all_probes_results_df["base_reference_alignment_region_hyphen"]
    != all_probes_results_df["base_reference_alignment_region_start_index"]
]

# Print the total number of discrepancies and agreements
total_discrepancies = discrepancies.shape[0]
total_agreements = all_probes_results_df.shape[0] - total_discrepancies

print(f"Total number of discrepancies: {total_discrepancies}")
print(f"Total number of agreements: {total_agreements}")

# Print detailed information for discrepancies
if total_discrepancies > 0:
    print("\nDiscrepancies found between the two methods:")
    for index, row in discrepancies.iterrows():
        print(f"\nDiscrepancy at index {index}:")
        print(f"Sample ID: {row['sample_id']}")
        print(f"Probe ID: {row['probe_id']}")
        print("Hyphen method sequence:")
        print(row["base_reference_alignment_region_hyphen"])
        print("Start index method sequence:")
        print(row["base_reference_alignment_region_start_index"])
        print("Base reference read position list:")
        print(row["base_reference_read_position_list"])
else:
    print("No discrepancies found. Both methods produce the same sequences.")

# Since there are no discrepancies between the two methods, we can merge the two columns into base_reference_alignment_region
all_probes_results_df["base_reference_alignment_region"] = all_probes_results_df[
    "base_reference_alignment_region_hyphen"
]

# Drop the old columns
all_probes_results_df.drop(
    columns=[
        "base_reference_alignment_region_hyphen",
        "base_reference_alignment_region_start_index",
    ],
    inplace=True,
)

# Sanity check: Verify the length of each sequence in 'base_reference_alignment_region'
lengths = all_probes_results_df["base_reference_alignment_region"].apply(len)
not_120_nt_long = all_probes_results_df[lengths != 120]

# Print the results of the sanity check
print("\nNumber of sequences not 120 nt long:", len(not_120_nt_long))
if len(not_120_nt_long) > 0:
    not_120_nt_long = not_120_nt_long.copy()
    not_120_nt_long["sequence_length"] = lengths[lengths != 120]
    print("Sequences not 120 nt long and their lengths:")
    print(not_120_nt_long[["sample_id", "probe_id", "sequence_length"]])
else:
    print("All sequences are 120 nt long.")

Total number of discrepancies: 0
Total number of agreements: 2355
No discrepancies found. Both methods produce the same sequences.

Number of sequences not 120 nt long: 27
Sequences not 120 nt long and their lengths:
                                   sample_id          probe_id  \
123   wtchgD00001556_closestdedup_vicunavfat  Geno1a_probe_124   
156   wtchgD00001556_closestdedup_vicunavfat  Geno1a_probe_157   
280   wtchgD00001562_closestdedup_vicunavfat  Geno1a_probe_124   
313   wtchgD00001562_closestdedup_vicunavfat  Geno1a_probe_157   
437   wtchgD00001561_closestdedup_vicunavfat  Geno1a_probe_124   
470   wtchgD00001561_closestdedup_vicunavfat  Geno1a_probe_157   
594   wtchgD00001576_closestdedup_vicunavfat  Geno1a_probe_124   
627   wtchgD00001576_closestdedup_vicunavfat  Geno1a_probe_157   
751   wtchgD00001577_closestdedup_vicunavfat  Geno1a_probe_124   
784   wtchgD00001577_closestdedup_vicunavfat  Geno1a_probe_157   
908   wtchgD00001567_closestdedup_vicunavfat  Geno1a_prob

### Genetic distance between probes and force aligned regions

#### Probe sequence vs. probe sequence from MSA

In [ ]:
# Create the new column 'probe_from_msa' by removing leading hyphens
all_probes_results_df["probe_from_msa"] = all_probes_results_df[
    "base_reference_msa_read_sequence"
].apply(lambda x: x.lstrip("-"))

# Find sequences in 'probe_from_msa' that contain hyphens (not just leading)
sequences_with_hyphens = all_probes_results_df[
    all_probes_results_df["probe_from_msa"].str.contains("-")
]

# Compare the 'probe_sequence' and 'probe_from_msa' columns to find discrepancies
discrepancies = all_probes_results_df[
    all_probes_results_df["probe_sequence"] != all_probes_results_df["probe_from_msa"]
]

# Check if all discrepancies are explained by hyphens in 'probe_from_msa'
all_explained_by_hyphens = True
# Print out the discrepancies if there are any
if not discrepancies.empty:
    print(f"Total number of discrepancies: {discrepancies.shape[0]}")
    print("Discrepancies found between 'probe_sequence' and 'probe_from_msa':")
    for index, row in discrepancies.iterrows():
        print(f"\nDiscrepancy at index {index}:")
        print(f"Sample ID: {row['sample_id']}")
        print(f"Probe ID: {row['probe_id']}")
        print(f"Probe sequence: {row['probe_sequence']}")
        print(f"Probe from MSA: {row['probe_from_msa']}")

        # Check if the discrepancy is explained by hyphens in 'probe_from_msa'
        if "-" in row["probe_from_msa"]:
            print("Discrepancy explained by hyphens in 'probe_from_msa'.")
        else:
            print("Discrepancy not explained by hyphens.")
else:
    print("No discrepancies found between 'probe_sequence' and 'probe_from_msa'.")

print("---")

if all_explained_by_hyphens:
    print("All discrepancies are explained by hyphens in 'probe_from_msa'.")
else:
    print("Not all discrepancies are explained by hyphens in 'probe_from_msa'.")

# Create a new column with hyphens removed from 'probe_from_msa'
all_probes_results_df["probe_from_msa_no_hyphens"] = all_probes_results_df[
    "probe_from_msa"
].str.replace("-", "")

# Compare the new column 'probe_from_msa_no_hyphens' with 'probe_sequence'
new_discrepancies = all_probes_results_df[
    all_probes_results_df["probe_sequence"]
    != all_probes_results_df["probe_from_msa_no_hyphens"]
]

# Check if there are any discrepancies after removing hyphens
if not new_discrepancies.empty:
    print(
        f"Total number of new discrepancies after removing hyphens: {new_discrepancies.shape[0]}"
    )
    print(
        "New discrepancies found between 'probe_sequence' and 'probe_from_msa_no_hyphens':"
    )
    for index, row in new_discrepancies.iterrows():
        print(f"\nDiscrepancy at index {index}:")
        print(f"Sample ID: {row['sample_id']}")
        print(f"Probe ID: {row['probe_id']}")
        print(f"Probe sequence: {row['probe_sequence']}")
        print(f"Probe from MSA without hyphens: {row['probe_from_msa_no_hyphens']}")
else:
    print(
        "No discrepancies found after removing hyphens. Both methods produce the same sequences."
    )

Total number of discrepancies: 39
Discrepancies found between 'probe_sequence' and 'probe_from_msa':

Discrepancy at index 628:
Sample ID: wtchgD00001577_closestdedup_vicunavfat
Probe ID: Geno1a_probe_1
Probe sequence: CCAGCCCCCTGATGGGGGCGACACTCCACCATGAATCACTCCCCTGTGAGGAACTACTGTCTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTGCAGCCTCCAGGACC
Probe from MSA: CCAGCCCCCTGATGGGGGCGACACTCCACCATGAATCACTCCCCTGTGAGGAACTACTGTCTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTGCAGCCTCCAGGA-CC
Discrepancy explained by hyphens in 'probe_from_msa'.

Discrepancy at index 629:
Sample ID: wtchgD00001577_closestdedup_vicunavfat
Probe ID: Geno1a_probe_2
Probe sequence: CTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTGCAGCCTCCAGGACCCCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTACACCGGAATTGCCA
Probe from MSA: CTTCACGCAGAAAGCGTCTAGCCATGGCGTTAGTATGAGTGTCGTGCAGCCTCCAGGACC-CCCCCCCTCCCGGGAGAGCCATAGTGGTCTGCGGAACCGGTGAGTACACCGGAATTGCCA
Discrepancy explained by hyphens in 'probe_from_msa'.

Discrepancy at index 630:
Sample ID:

#### Comparing lengths of probe sequences and alignment regions
There are only 8 probe-target region pairs with differing lengths. I will just ignore those.

In [ ]:
# Compare the lengths of 'probe_sequence' and 'base_reference_alignment_region'
length_discrepancies = all_probes_results_df[
    all_probes_results_df["probe_sequence"].apply(len)
    != all_probes_results_df["base_reference_alignment_region"].apply(len)
]

# Print out the discrepancies if there are any
if not length_discrepancies.empty:
    print(f"Total number of length discrepancies: {length_discrepancies.shape[0]}")
    print(
        "Length discrepancies found between 'probe_sequence' and 'base_reference_alignment_region':"
    )
    for index, row in length_discrepancies.iterrows():
        print(f"\nDiscrepancy at index {index}:")
        print(f"Sample ID: {row['sample_id']}")
        print(f"Probe ID: {row['probe_id']}")
        print(f"Probe sequence length: {len(row['probe_sequence'])}")
        print(
            f"Base reference alignment region length: {len(row['base_reference_alignment_region'])}"
        )
else:
    print(
        "No length discrepancies found between 'probe_sequence' and 'base_reference_alignment_region'."
    )

Total number of length discrepancies: 8
Length discrepancies found between 'probe_sequence' and 'base_reference_alignment_region':

Discrepancy at index 1693:
Sample ID: wtchgD00001559_closestdedup_vicunavfat
Probe ID: Geno1a_probe_124
Probe sequence length: 108
Base reference alignment region length: 117

Discrepancy at index 1850:
Sample ID: wtchgD00001557_closestdedup_vicunavfat
Probe ID: Geno1a_probe_124
Probe sequence length: 108
Base reference alignment region length: 117

Discrepancy at index 2007:
Sample ID: wtchgD00001571_closestdedup_vicunavfat
Probe ID: Geno1a_probe_124
Probe sequence length: 108
Base reference alignment region length: 120

Discrepancy at index 2040:
Sample ID: wtchgD00001571_closestdedup_vicunavfat
Probe ID: Geno1a_probe_157
Probe sequence length: 85
Base reference alignment region length: 88

Discrepancy at index 2164:
Sample ID: wtchgD00001583_closestdedup_vicunavfat
Probe ID: Geno1a_probe_124
Probe sequence length: 108
Base reference alignment region len

#### Genetic distance, ignoring rows with differing lengths and too many N characters

In [ ]:
# Function to count the number of 'N' characters in a sequence
def count_n_characters(seq):
    return seq.count("N")


# Apply the count function to each row and store the results in a new column
all_probes_results_df["num_n_characters"] = all_probes_results_df[
    "base_reference_alignment_region"
].apply(count_n_characters)

# Filter out sequences where lengths of 'probe_sequence' and 'base_reference_alignment_region' are different
# and where 'num_n_characters' is greater than 20
valid_length_df = all_probes_results_df[
    (
        all_probes_results_df["probe_sequence"].apply(len)
        == all_probes_results_df["base_reference_alignment_region"].apply(len)
    )
    & (all_probes_results_df["num_n_characters"] <= 1)
]


# Function to calculate genetic distance between two sequences, ignoring hyphens
def calculate_genetic_distance(seq1, seq2):
    """
    Calculate the genetic distance between two sequences, ignoring hyphens.

    Parameters:
    seq1 (str): First sequence.
    seq2 (str): Second sequence.

    Returns:
    float: The percentage of similarity between the sequences.
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be of equal length.")

    matches = 0
    valid_positions = 0

    for a, b in zip(seq1, seq2):
        if a != "-" and b != "-":
            valid_positions += 1
            if a == b:
                matches += 1

    if valid_positions == 0:
        return float("nan")  # If there are no valid positions, return NaN

    distance = matches / valid_positions
    return distance


# Apply the genetic distance calculation to each row and store the results in a new column
valid_length_df["genetic_distance"] = valid_length_df.apply(
    lambda row: calculate_genetic_distance(
        row["probe_sequence"], row["base_reference_alignment_region"]
    ),
    axis=1,
)

# Merge the results back into the original DataFrame
all_probes_results_df = all_probes_results_df.merge(
    valid_length_df[["sample_id", "probe_id", "genetic_distance"]],
    on=["sample_id", "probe_id"],
    how="left",
)

/var/folders/th/lqlr4bd14y70d5dlhczfj2v40000gn/T/ipykernel_70225/3995797635.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_length_df["genetic_distance"] = valid_length_df.apply(


#### Are lists in base_reference_read_position_list all continuous?
Only 39 lists are non-continuous. Based on eyeballing the data, this usually just means that one integer is skipped. In practice this means that one read depth value might either be missing or different from what it should be once I extract the coverage values based on the base_reference_read_position_list values as indexes. This should only be a minor issue.

In [ ]:
def find_non_continuous_positions(lst):
    # Remove leading None values
    trimmed_lst = [x for x in lst if x is not None]

    non_continuous_positions = []
    for i in range(1, len(trimmed_lst)):
        if trimmed_lst[i] != trimmed_lst[i - 1] + 1:
            non_continuous_positions.append(trimmed_lst[i])
    return non_continuous_positions


# Apply the continuity check to each row and store the results in a new column
all_probes_results_df["non_continuous_positions"] = all_probes_results_df[
    "base_reference_read_position_list"
].apply(find_non_continuous_positions)

# Filter rows where the lists are not continuous
non_continuous_rows = all_probes_results_df[
    all_probes_results_df["non_continuous_positions"].apply(len) > 0
]

# Print out the results
print(f"Total number of non-continuous lists: {non_continuous_rows.shape[0]}")
if not non_continuous_rows.empty:
    print("Non-continuous lists found in 'base_reference_read_position_list':")
    for index, row in non_continuous_rows.iterrows():
        print(f"\nNon-continuous list at index {index}:")
        print(f"Sample ID: {row['sample_id']}")
        print(f"Probe ID: {row['probe_id']}")
        print(
            f"Base reference read position list: {row['base_reference_read_position_list']}"
        )
        print(f"Non-continuous positions: {row['non_continuous_positions']}")
else:
    print("All lists in 'base_reference_read_position_list' are continuous.")

Total number of non-continuous lists: 39
Non-continuous lists found in 'base_reference_read_position_list':

Non-continuous list at index 628:
Sample ID: wtchgD00001577_closestdedup_vicunavfat
Probe ID: Geno1a_probe_1
Base reference read position list: [None, None, None, None, None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]
Non-continuous positions: [115]

Non-continuous list at index 629:
Sample ID: wtchgD00001577_closestdedup_vicunavfat
Probe ID: Geno1a_probe_2
Base reference read position list: [56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,

### Coverage information

In [ ]:
# Rename 'sample_id' to 'run_id'
all_probes_results_df.rename(columns={"sample_id": "run_id"}, inplace=True)


# Strip suffix function
def strip_suffix(run_id):
    return run_id.split("_closestdedup_vicunavfat")[0]


# Apply the function to the 'run_id' column
all_probes_results_df["run_id"] = all_probes_results_df["run_id"].apply(strip_suffix)


#### One example first, counting removing None values

In [ ]:
# Example: Get coverage information for the row at index 0 in all_probes_results_df
index = 0
probe_row = all_probes_results_df.loc[index]

# Find the corresponding sample in high_VL_samples_filtered
run_id = probe_row["run_id"]
sample_row = high_VL_samples_filtered[high_VL_samples_filtered["run_id"] == run_id]

if not sample_row.empty:
    # Get the coverage list from the sample row
    coverage_list = sample_row.iloc[0]["coverage"]

    # Get the base reference read position list from the probe row
    base_reference_read_position_list = probe_row["base_reference_read_position_list"]

    # Count the number of None values at the beginning
    none_count = 0
    for pos in base_reference_read_position_list:
        if pos is None:
            none_count += 1
        else:
            break

    # Extract the corresponding coverage values, skipping the None values at the beginning
    corresponding_coverage_values = [
        coverage_list[pos - 1]
        for pos in base_reference_read_position_list[none_count:]
        if pos is not None and pos - 1 < len(coverage_list)
    ]

    # Subtract the number of None values from the beginning
    corresponding_coverage_values = corresponding_coverage_values[none_count:]

    # Perform sanity check: Compare the lengths of base_reference_read_position_list and corresponding_coverage_values
    list_length = len(base_reference_read_position_list) - none_count
    coverage_length = len(corresponding_coverage_values)

    # Print the results
    print(f"Run ID: {run_id}")
    print(
        f"Base reference read position list (length {list_length}): {base_reference_read_position_list}"
    )
    print(
        f"Corresponding coverage values (length {coverage_length}): {corresponding_coverage_values}"
    )

    # Sanity check result
    if list_length == coverage_length:
        print("Sanity check passed: The lengths of the lists are the same.")
    else:
        print("Sanity check failed: The lengths of the lists are different.")
else:
    print(f"No sample found for Run ID: {run_id}")

Run ID: wtchgD00001556
Base reference read position list (length 114): [None, None, None, None, None, None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]
Corresponding coverage values (length 108): [2, 2, 2, 2, 2, 10, 12, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 20, 21, 21, 21, 21, 21, 21, 23, 23, 23, 24, 24, 25, 25, 30, 31, 31, 32, 32, 32, 32, 34, 35, 35, 35, 35, 35, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 38, 41, 43, 43, 43, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43

#### Example, retaining None values

In [ ]:
# Example: Get coverage information for the row at index 0 in all_probes_results_df
index = 0
probe_row = all_probes_results_df.loc[index]

# Find the corresponding sample in high_VL_samples_filtered
run_id = probe_row["run_id"]
sample_row = high_VL_samples_filtered[high_VL_samples_filtered["run_id"] == run_id]

if not sample_row.empty:
    # Get the coverage list from the sample row
    coverage_list = sample_row.iloc[0]["coverage"]

    # Get the base reference read position list from the probe row
    base_reference_read_position_list = probe_row["base_reference_read_position_list"]

    # Extract the corresponding coverage values, skipping the None values
    corresponding_coverage_values = [
        coverage_list[pos - 1]
        for pos in base_reference_read_position_list
        if pos is not None and pos - 1 < len(coverage_list)  # get rid of this none?
    ]

    # Perform sanity check: Compare the lengths of base_reference_read_position_list (excluding None) and corresponding_coverage_values
    list_length = len(
        [pos for pos in base_reference_read_position_list if pos is not None]
    )
    coverage_length = len(corresponding_coverage_values)

    # Print the results
    print(f"Run ID: {run_id}")
    print(
        f"Base reference read position list (length {list_length}): {base_reference_read_position_list}"
    )
    print(
        f"Corresponding coverage values (length {coverage_length}): {corresponding_coverage_values}"
    )

    # Sanity check result
    if list_length == coverage_length:
        print("Sanity check passed: The lengths of the lists are the same.")
    else:
        print("Sanity check failed: The lengths of the lists are different.")
else:
    print(f"No sample found for Run ID: {run_id}")

Run ID: wtchgD00001556
Base reference read position list (length 114): [None, None, None, None, None, None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]
Corresponding coverage values (length 114): [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 12, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 20, 21, 21, 21, 21, 21, 21, 23, 23, 23, 24, 24, 25, 25, 30, 31, 31, 32, 32, 32, 32, 34, 35, 35, 35, 35, 35, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 38, 41, 43, 43, 43, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 

#### HiSeq and MiSeq coverage lists
First, check whether the lengths of the coverage lists for the HiSeq and MiSeq runs for each sample are the same. Thankfully, they are.

In [ ]:
# Initialize a list to collect any sample_id with coverage length discrepancies
coverage_length_discrepancies = []

# Group by sample_id
grouped_samples = high_VL_samples_filtered.groupby("sample_id")

# Iterate over each group
for sample_id, group in grouped_samples:
    # Ensure there are exactly two rows for each sample_id
    if len(group) == 2:
        coverage_lengths = group["coverage"].apply(len).tolist()

        # Check if the lengths are the same
        if coverage_lengths[0] != coverage_lengths[1]:
            coverage_length_discrepancies.append(
                (sample_id, coverage_lengths[0], coverage_lengths[1])
            )
    else:
        print(f"Sample ID {sample_id} does not have exactly two rows.")

# Print the results of the coverage length check
if coverage_length_discrepancies:
    print(
        f"Total number of sample_ids with coverage length discrepancies: {len(coverage_length_discrepancies)}"
    )
    for discrepancy in coverage_length_discrepancies:
        print(
            f"Sample ID: {discrepancy[0]}, rep_2_precapture length = {discrepancy[1]}, rep_2_G1a_capture length = {discrepancy[2]}"
        )
else:
    print(
        "All sample_ids have matching lengths of the coverage lists for the pre- and post-capture sequencing runs."
    )

All sample_ids have matching lengths of the coverage lists for the pre- and post-capture sequencing runs.


#### Example coverage lists for one pair of HiSeq and MiSeq runs

In [ ]:
# Example: Get coverage information for the row at index 0 in all_probes_results_df
index = 0
probe_row = all_probes_results_df.loc[index]

# Find the corresponding sample in high_VL_samples_filtered
run_id = probe_row["run_id"]
sample_row = high_VL_samples_filtered[high_VL_samples_filtered["run_id"] == run_id]

if not sample_row.empty:
    sample_id = sample_row.iloc[0]["sample_id"]

    # Ensure the seq_method is "rep_2_precapture"
    if sample_row.iloc[0]["seq_method"] == "rep_2_precapture":
        # Get the coverage list for HiSeq (rep_2_precapture)
        coverage_list_hiseq = sample_row.iloc[0]["coverage"]

        # Find the corresponding MiSeq (rep_2_G1a_capture) row
        miseq_row = high_VL_samples_filtered[
            (high_VL_samples_filtered["sample_id"] == sample_id)
            & (high_VL_samples_filtered["seq_method"] == "rep_2_G1a_capture")
        ]

        if not miseq_row.empty:
            # Get the coverage list for MiSeq (rep_2_G1a_capture)
            coverage_list_miseq = miseq_row.iloc[0]["coverage"]

            # Get the base reference read position list from the probe row
            base_reference_read_position_list = probe_row[
                "base_reference_read_position_list"
            ]

            # Extract the corresponding coverage values for HiSeq, skipping None values
            corresponding_coverage_values_hiseq = [
                coverage_list_hiseq[pos - 1]
                for pos in base_reference_read_position_list
                if pos is not None and pos - 1 < len(coverage_list_hiseq)
            ]

            # Extract the corresponding coverage values for MiSeq, skipping None values
            corresponding_coverage_values_miseq = [
                coverage_list_miseq[pos - 1]
                for pos in base_reference_read_position_list
                if pos is not None and pos - 1 < len(coverage_list_miseq)
            ]

            # Perform sanity check: Compare the lengths of base_reference_read_position_list (excluding None) and corresponding_coverage_values
            list_length = len(
                [pos for pos in base_reference_read_position_list if pos is not None]
            )
            hiseq_coverage_length = len(corresponding_coverage_values_hiseq)
            miseq_coverage_length = len(corresponding_coverage_values_miseq)

            # Print the results
            print(f"Run ID: {run_id}")
            print(
                f"Base reference read position list (length {list_length}): {base_reference_read_position_list}"
            )
            print(
                f"Corresponding HiSeq coverage values (length {hiseq_coverage_length}): {corresponding_coverage_values_hiseq}"
            )
            print(
                f"Corresponding MiSeq coverage values (length {miseq_coverage_length}): {corresponding_coverage_values_miseq}"
            )

            # Sanity check result
            if list_length == hiseq_coverage_length == miseq_coverage_length:
                print("Sanity check passed: The lengths of the lists are the same.")
            else:
                print("Sanity check failed: The lengths of the lists are different.")
        else:
            print(f"No MiSeq sample found for Sample ID: {sample_id}")
    else:
        print(
            f"The sample does not correspond to seq_method 'rep_2_precapture'. Run ID: {run_id}"
        )
else:
    print(f"No sample found for Run ID: {run_id}")

Run ID: wtchgD00001556
Base reference read position list (length 114): [None, None, None, None, None, None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]
Corresponding HiSeq coverage values (length 114): [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 12, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 20, 21, 21, 21, 21, 21, 21, 23, 23, 23, 24, 24, 25, 25, 30, 31, 31, 32, 32, 32, 32, 34, 35, 35, 35, 35, 35, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 38, 41, 43, 43, 43, 41, 41, 41, 41, 41, 42, 42, 42, 42

#### All probe-target pairs coverage lists for HiSeq and MiSeq runs

In [ ]:
# Generalize to process all rows in all_probes_results_df
coverage_region_hiseq_list = []
coverage_region_miseq_list = []
length_discrepancies = []

for index, probe_row in all_probes_results_df.iterrows():
    run_id = probe_row["run_id"]
    sample_row = high_VL_samples_filtered[high_VL_samples_filtered["run_id"] == run_id]

    if not sample_row.empty:
        sample_id = sample_row.iloc[0]["sample_id"]

        # Ensure the seq_method is "rep_2_precapture"
        if sample_row.iloc[0]["seq_method"] == "rep_2_precapture":
            # Get the coverage list for HiSeq (rep_2_precapture)
            coverage_list_hiseq = sample_row.iloc[0]["coverage"]

            # Find the corresponding MiSeq (rep_2_G1a_capture) row
            miseq_row = high_VL_samples_filtered[
                (high_VL_samples_filtered["sample_id"] == sample_id)
                & (high_VL_samples_filtered["seq_method"] == "rep_2_G1a_capture")
            ]

            if not miseq_row.empty:
                # Get the coverage list for MiSeq (rep_2_G1a_capture)
                coverage_list_miseq = miseq_row.iloc[0]["coverage"]

                # Get the base reference read position list from the probe row
                base_reference_read_position_list = probe_row[
                    "base_reference_read_position_list"
                ]

                # Extract the corresponding coverage values for HiSeq, skipping None values
                corresponding_coverage_values_hiseq = [
                    coverage_list_hiseq[pos - 1]
                    for pos in base_reference_read_position_list
                    if pos is not None and pos - 1 < len(coverage_list_hiseq)
                ]

                # Extract the corresponding coverage values for MiSeq, skipping None values
                corresponding_coverage_values_miseq = [
                    coverage_list_miseq[pos - 1]
                    for pos in base_reference_read_position_list
                    if pos is not None and pos - 1 < len(coverage_list_miseq)
                ]

                # Perform sanity check: Compare the lengths of base_reference_read_position_list (excluding None) and corresponding_coverage_values
                list_length = len(
                    [
                        pos
                        for pos in base_reference_read_position_list
                        if pos is not None
                    ]
                )
                hiseq_coverage_length = len(corresponding_coverage_values_hiseq)
                miseq_coverage_length = len(corresponding_coverage_values_miseq)

                # If lengths do not match, add the index to length_discrepancies
                if (
                    list_length != hiseq_coverage_length
                    or list_length != miseq_coverage_length
                ):
                    length_discrepancies.append(
                        (
                            index,
                            list_length,
                            hiseq_coverage_length,
                            miseq_coverage_length,
                        )
                    )

                # Append the coverage values to the respective lists
                coverage_region_hiseq_list.append(corresponding_coverage_values_hiseq)
                coverage_region_miseq_list.append(corresponding_coverage_values_miseq)
            else:
                print(f"No MiSeq sample found for Sample ID: {sample_id}")
                coverage_region_hiseq_list.append(None)
                coverage_region_miseq_list.append(None)
        else:
            print(
                f"The sample does not correspond to seq_method 'rep_2_precapture'. Run ID: {run_id}"
            )
            coverage_region_hiseq_list.append(None)
            coverage_region_miseq_list.append(None)
    else:
        print(f"No sample found for Run ID: {run_id}")
        coverage_region_hiseq_list.append(None)
        coverage_region_miseq_list.append(None)

# Add the coverage lists to the DataFrame
all_probes_results_df["coverage_region_hiseq"] = coverage_region_hiseq_list
all_probes_results_df["coverage_region_miseq"] = coverage_region_miseq_list

# Print the results of the sanity check
if length_discrepancies:
    print(f"Total number of length discrepancies: {len(length_discrepancies)}")
    for discrepancy in length_discrepancies:
        print(
            f"Discrepancy at index {discrepancy[0]}: List length = {discrepancy[1]}, HiSeq coverage length = {discrepancy[2]}, MiSeq coverage length = {discrepancy[3]}"
        )
else:
    print("Sanity check passed: All lists have matching lengths.")

Sanity check passed: All lists have matching lengths.


### Mean coverages and normalising
The lengths of the forced alignment regions and the lengths of coverage value lists differ in some instances. This is probably because of hyphens or None values or something but I won't look into it for now. The question is around which "alignment length" value to take to divide the mean coverages by

In [ ]:
# Add length_of_base_reference_alignment_region column
all_probes_results_df["length_of_base_reference_alignment_region"] = (
    all_probes_results_df["base_reference_alignment_region"].apply(len)
)

# Add a column 'length_of_coverage_region' that contains the lengths of the coverage lists in 'coverage_region_hiseq'
all_probes_results_df["length_of_coverage_region"] = all_probes_results_df[
    "coverage_region_hiseq"
].apply(len)

# Check if the values in length_of_base_reference_alignment_region are all the same as the lengths of the lists in coverage_region_hiseq
length_check = (
    all_probes_results_df["length_of_base_reference_alignment_region"]
    == all_probes_results_df["length_of_coverage_region"]
)

# Print the result of the check
if length_check.all():
    print(
        "All values in length_of_base_reference_alignment_region are the same as the lengths of the lists in coverage_region_hiseq."
    )
else:
    print(
        "There are discrepancies between length_of_base_reference_alignment_region and the lengths of the lists in coverage_region_hiseq."
    )
    discrepancies = all_probes_results_df[~length_check]
    print(
        discrepancies[
            [
                "run_id",
                "probe_id",
                "length_of_base_reference_alignment_region",
                "length_of_coverage_region",
            ]
        ]
    )

There are discrepancies between length_of_base_reference_alignment_region and the lengths of the lists in coverage_region_hiseq.
              run_id          probe_id  \
0     wtchgD00001556    Geno1a_probe_1   
1     wtchgD00001556    Geno1a_probe_2   
23    wtchgD00001556   Geno1a_probe_24   
24    wtchgD00001556   Geno1a_probe_25   
25    wtchgD00001556   Geno1a_probe_26   
...              ...               ...   
2337  wtchgD00001582  Geno1a_probe_140   
2346  wtchgD00001582  Geno1a_probe_149   
2347  wtchgD00001582  Geno1a_probe_150   
2353  wtchgD00001582  Geno1a_probe_156   
2354  wtchgD00001582  Geno1a_probe_157   

      length_of_base_reference_alignment_region  length_of_coverage_region  
0                                           120                        114  
1                                           120                        118  
23                                          120                        111  
24                                          120           

### Normalise per sample instead

In [ ]:
# Calculate mean coverage for HiSeq and MiSeq and store in new columns
all_probes_results_df["mean_coverage_hiseq"] = all_probes_results_df[
    "coverage_region_hiseq"
].apply(lambda x: sum(x) / len(x) if len(x) > 0 else 0)
all_probes_results_df["mean_coverage_miseq"] = all_probes_results_df[
    "coverage_region_miseq"
].apply(lambda x: sum(x) / len(x) if len(x) > 0 else 0)

# Calculate coverage ratio
all_probes_results_df["coverage_ratio"] = (
    all_probes_results_df["mean_coverage_miseq"]
    / all_probes_results_df["mean_coverage_hiseq"]
)

# Find the maximum value in coverage_ratio
max_coverage_ratio = all_probes_results_df["coverage_ratio"].max()

# Normalize coverage_ratio values by the maximum coverage_ratio value
all_probes_results_df["normalized_coverage"] = (
    all_probes_results_df["coverage_ratio"] / max_coverage_ratio
)


def normalize_per_sample(group):
    max_ratio = group["coverage_ratio"].max()
    return group["coverage_ratio"] / max_ratio


all_probes_results_df["normalized_coverage_per_sample"] = all_probes_results_df.groupby(
    "run_id"
)["coverage_ratio"].transform(lambda x: x / x.max())

## GC content calculation

In [ ]:
def calculate_gc_content(sequence):
    """Calculate the GC content of a given sequence."""
    gc_count = sequence.count("G") + sequence.count("C")
    total_count = len(sequence)
    return (gc_count / total_count) * 100 if total_count > 0 else 0


def calculate_complementary_gc_content(subject_seq, query_seq):
    """Calculate the GC content only for complementary positions in two sequences."""
    gc_count = sum(
        1 for s, q in zip(subject_seq, query_seq) if s == q and s in ["G", "C"]
    )
    total_count = sum(1 for s, q in zip(subject_seq, query_seq) if s == q)
    return (gc_count / total_count) * 100 if total_count > 0 else 0


# Calculate GC content for base_reference_alignment_region (target region)
all_probes_results_df["gc_alignment_region"] = all_probes_results_df[
    "base_reference_alignment_region"
].apply(calculate_gc_content)

# Calculate GC content for the probe sequences
all_probes_results_df["gc_probe_sequence"] = all_probes_results_df[
    "probe_sequence"
].apply(calculate_gc_content)

# Calculate GC content for matching positions in probe and target sequence
all_probes_results_df["gc_complementary"] = all_probes_results_df.apply(
    lambda row: calculate_complementary_gc_content(
        row["base_reference_alignment_region"], row["probe_sequence"]
    ),
    axis=1,
)

## Hydrogen bond score calculation

In [ ]:
def calculate_hydrogen_bond_score(ref_seq, probe_seq):
    """
    Calculate hydrogen bond score based on matching positions in the alignment.

    Parameters:
    ref_seq (str): Reference sequence (base_reference_alignment_region).
    probe_seq (str): Probe sequence.

    Returns:
    int: Hydrogen bond score based on matching positions.
    """
    hydrogen_bond_score = 0

    for ref_base, probe_base in zip(ref_seq, probe_seq):
        if ref_base == probe_base:
            if ref_base in "AT":
                hydrogen_bond_score += 2
            elif ref_base in "GC":
                hydrogen_bond_score += 3

    return hydrogen_bond_score


# Calculate hydrogen bond score
all_probes_results_df["hydrogen_bond_score"] = all_probes_results_df.apply(
    lambda row: calculate_hydrogen_bond_score(
        row["base_reference_alignment_region"], row["probe_sequence"]
    ),
    axis=1,
)

print(all_probes_results_df.columns)

Index(['run_id', 'probe_id', 'probe_sequence',
       'matched_reference_position_list', 'reference_msa_position_list',
       'base_reference_read_position_list', 'base_reference_msa_read_sequence',
       'base_reference_msa_reference_sequence',
       'base_reference_alignment_region', 'probe_from_msa',
       'probe_from_msa_no_hyphens', 'num_n_characters', 'genetic_distance',
       'non_continuous_positions', 'coverage_region_hiseq',
       'coverage_region_miseq', 'length_of_base_reference_alignment_region',
       'length_of_coverage_region', 'mean_coverage_hiseq',
       'mean_coverage_miseq', 'coverage_ratio', 'normalized_coverage',
       'normalized_coverage_per_sample', 'gc_alignment_region',
       'gc_probe_sequence', 'gc_complementary', 'hydrogen_bond_score'],
      dtype='object')


## Retain viral load etc. information

In [ ]:
# Create a DataFrame for HiSeq data from high_VL_samples_filtered
high_VL_samples_hiseq = high_VL_samples_filtered[
    high_VL_samples_filtered["seq_method"] == "rep_2_precapture"
].copy()

# No need to rename columns with suffixes since we're only dealing with HiSeq data
# Just drop the 'seq_method' column as it's no longer needed
high_VL_samples_hiseq = high_VL_samples_hiseq.drop(columns=["seq_method"])

# Merge all_probes_results_df with high_VL_samples_hiseq
all_probes_results_df = all_probes_results_df.merge(
    high_VL_samples_hiseq, on="run_id", how="left"
)

# If 'coverage' column exists and you don't need it, drop it
if "coverage" in all_probes_results_df.columns:
    all_probes_results_df = all_probes_results_df.drop(columns=["coverage"])

# Print the new column names to verify
print(all_probes_results_df.columns)

Index(['run_id', 'probe_id', 'probe_sequence',
       'matched_reference_position_list', 'reference_msa_position_list',
       'base_reference_read_position_list', 'base_reference_msa_read_sequence',
       'base_reference_msa_reference_sequence',
       'base_reference_alignment_region', 'probe_from_msa',
       'probe_from_msa_no_hyphens', 'num_n_characters', 'genetic_distance',
       'non_continuous_positions', 'coverage_region_hiseq',
       'coverage_region_miseq', 'length_of_base_reference_alignment_region',
       'length_of_coverage_region', 'mean_coverage_hiseq',
       'mean_coverage_miseq', 'coverage_ratio', 'normalized_coverage',
       'normalized_coverage_per_sample', 'gc_alignment_region',
       'gc_probe_sequence', 'gc_complementary', 'hydrogen_bond_score',
       'sample_id', 'viral_load', 'closest_ref', 'q50_major_type'],
      dtype='object')


## Comparing with earlier capture efficiency plot

In [ ]:
# Path to your BLAST XML output file
xml_file = "../blast_high_VL_samples/blast_output/2024-05-16_Geno1a_probes_queried_against_refs_and_assemblies_4C4B42GA114-Alignment.xml"

# Parse the BLAST XML output
blast_results_df_original = parse_blast_xml_to_df(xml_file)

# DataFrame to store combined results for all runs
hiseq_and_miseq_alignment_info_df = pd.DataFrame()

# Process HiSeq (rep_2_precapture) and MiSeq (rep_2_G1a_capture) runs separately
for seq_method in ["rep_2_precapture", "rep_2_G1a_capture"]:
    # Filter runs by sequence method
    run_ids = high_VL_samples_filtered[
        high_VL_samples_filtered["seq_method"] == seq_method
    ]["run_id"].unique()

    for run_id in run_ids:
        # Extract coverage information for the current run_id
        coverage_info = None
        sample_id = None
        for _, row in high_VL_samples_filtered.iterrows():
            if row["run_id"] == run_id:
                coverage_info = row["coverage"]
                sample_id = row["sample_id"]
                break

        if coverage_info is None:
            continue  # Skip if no coverage information is found

        # Find all alignment information for the current run_id in blast_results_df_original
        alignments_info_df = blast_results_df_original[
            blast_results_df_original["hit_def"].apply(strip_suffix) == run_id
        ].copy()
        if alignments_info_df.empty:
            continue  # Skip if no alignment information is found

        # Calculate and store mean coverages and coverage regions as new columns in alignments_info_df
        alignments_info_df[["mean_coverage", "coverage_values_probe_region"]] = (
            alignments_info_df.apply(
                lambda row: pd.Series(
                    calculate_mean_coverage_and_region(
                        coverage_info, row["align_start"], row["align_len"]
                    )
                ),
                axis=1,
            )
        )

        # Add sample_id, run_id, and seq_method columns to alignments_info_df
        alignments_info_df["sample_id"] = sample_id
        alignments_info_df["run_id"] = run_id
        alignments_info_df["seq_method"] = seq_method

        # Append alignments_info_df to the combined DataFrame
        hiseq_and_miseq_alignment_info_df = pd.concat(
            [hiseq_and_miseq_alignment_info_df, alignments_info_df], ignore_index=True
        )

# Separate the DataFrame into HiSeq and MiSeq data
hiseq_df = hiseq_and_miseq_alignment_info_df[
    hiseq_and_miseq_alignment_info_df["seq_method"] == "rep_2_precapture"
].copy()
miseq_df = hiseq_and_miseq_alignment_info_df[
    hiseq_and_miseq_alignment_info_df["seq_method"] == "rep_2_G1a_capture"
].copy()

# Rename columns to prepare for merging
hiseq_df = hiseq_df.rename(
    columns={
        "run_id": "run_id_hiseq",
        "number_of_matches": "number_of_matches_hiseq",
        "num_sims_per_base": "num_sims_per_base_hiseq",
        "subject_alignment_seq": "subject_alignment_hiseq",
        "query_alignment_seq": "query_alignment_hiseq",
        "align_len": "align_len_hiseq",
        "align_start": "align_start_hiseq",
        "align_gap": "align_gap_hiseq",
        "mean_coverage": "mean_coverage_hiseq",
        "coverage_values_probe_region": "coverage_values_probe_region_hiseq",
    }
)

miseq_df = miseq_df.rename(
    columns={
        "run_id": "run_id_miseq",
        "number_of_matches": "number_of_matches_miseq",
        "num_sims_per_base": "num_sims_per_base_miseq",
        "subject_alignment_seq": "subject_alignment_miseq",
        "query_alignment_seq": "query_alignment_miseq",
        "align_len": "align_len_miseq",
        "align_start": "align_start_miseq",
        "align_gap": "align_gap_miseq",
        "mean_coverage": "mean_coverage_miseq",
        "coverage_values_probe_region": "coverage_values_probe_region_miseq",
    }
)

# Filter the dataframes to only include probe alignments longer than 20 because there a handful of very short alignments of only 16 nucleotides that have big outlier values for # the coverage_ratio
hiseq_df = hiseq_df[hiseq_df["align_len_hiseq"] > 20]
miseq_df = miseq_df[miseq_df["align_len_miseq"] > 20]

# Merge the HiSeq and MiSeq DataFrames on 'sample_id' and 'query_def'
all_high_VL_samples_alignment_info_df = pd.merge(
    hiseq_df,
    miseq_df,
    on=["sample_id", "query_def"],
    suffixes=("_hiseq", "_miseq"),
    how="inner",
)

# Drop the seq_method columns as they are no longer needed
all_high_VL_samples_alignment_info_df = all_high_VL_samples_alignment_info_df.drop(
    columns=["seq_method_hiseq", "seq_method_miseq"]
)

# Create a temporary DataFrame to hold unique sample_id and viral_load
unique_sample_viral_load_df = high_VL_samples_filtered[
    ["sample_id", "viral_load"]
].drop_duplicates()

# Merge with all_high_VL_samples_alignment_info_df to add the viral_load column
all_high_VL_samples_alignment_info_df = pd.merge(
    all_high_VL_samples_alignment_info_df,
    unique_sample_viral_load_df,
    on="sample_id",
    how="left",
)

# Calculate coverage_ratio by dividing mean_coverage_miseq by mean_coverage_hiseq
all_high_VL_samples_alignment_info_df["coverage_ratio"] = (
    all_high_VL_samples_alignment_info_df["mean_coverage_miseq"]
    / all_high_VL_samples_alignment_info_df["mean_coverage_hiseq"]
)

# Find the maximum coverage_ratio value
max_coverage_ratio = all_high_VL_samples_alignment_info_df["coverage_ratio"].max()

# Calculate normalized_coverage by dividing each coverage_ratio value by the maximum coverage_ratio value
all_high_VL_samples_alignment_info_df["normalized_coverage"] = (
    all_high_VL_samples_alignment_info_df["coverage_ratio"] / max_coverage_ratio
)

## Save dataframe to share with colleagues

In [ ]:
# Export to TSV
all_probes_results_df.to_csv(
    "../data_exports/forced_alignments_and_analysis_results_dataframe.tsv",
    sep="\t",
    index=False,
)

# # Export to JSON with indentation for better readability:
# all_probes_results_df.to_json(
#     "forced_alignments_and_analysis_results_dataframe.json", orient="records", indent=2
# )

# Export to TSV
all_high_VL_samples_alignment_info_df.to_csv(
    "../data_exports/initial_BLAST_alignments_dataframe_all_high_VL_samples_alignment_info_df.tsv",
    sep="\t",
    index=False,
)